In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import pdist, squareform


node_data = pd.read_csv('rank.csv')
relation_data = pd.read_csv('rel.csv')
relation_data['line_id'] = range(len(relation_data))

# 初始化每个网格，每个网格中存储经过的线条的集合
grid_size = 700
grid_step = 1
grid_vectors = np.empty((grid_size // grid_step, grid_size // grid_step), dtype=object)
for i in range(grid_size // grid_step):
    for j in range(grid_size // grid_step):
        grid_vectors[i, j] = set()


# 圆心与半径
center_x, center_y = 950, 500
radius = 401

# 得到每个点的极坐标，然后筛选在圆内的点
node_data['r'] = np.sqrt((node_data['x'] - center_x) ** 2 + (node_data['y'] - center_y) ** 2)
node_data['theta'] = np.arctan2(node_data['y'] - center_y, node_data['x'] - center_x)

node_data = node_data[node_data['r'] <= radius]

# 原始坐标范围转换为0，2*radius区间内，然后归一化映射到网格中
node_data['grid_x'] = ((node_data['x'] - (center_x - radius)) / (2 * radius) * (grid_size - 1)).astype(int)
node_data['grid_y'] = ((node_data['y'] - (center_y - radius)) / (2 * radius) * (grid_size - 1)).astype(int)
node_dict = node_data.set_index('id')[['grid_x', 'grid_y']].to_dict(orient='index')




# 遍历所有的关系线条
for _, row in relation_data.iterrows():
    source_id = row['source']
    target_id = row['target']
    line_id = row['line_id']
    
    if source_id in node_dict and target_id in node_dict:
        x0, y0 = node_dict[source_id]['grid_x'], node_dict[source_id]['grid_y']
        x1, y1 = node_dict[target_id]['grid_x'], node_dict[target_id]['grid_y']
        
        # Bresenham算法计算经过的网格
        dx = abs(x1 - x0)
        dy = abs(y1 - y0)
        sx = 1 if x0 < x1 else -1
        sy = 1 if y0 < y1 else -1
        err = dx - dy

        while True:
            grid_vectors[y0 // grid_step, x0 // grid_step].add(line_id)
            if x0 == x1 and y0 == y1:
                break
            e2 = 2 * err
            if e2 > -dy:
                err -= dy
                x0 += sx
            if e2 < dx:
                err += dx
                y0 += sy


# 网格采样，高密度区域和低密度区域的采样

# 获取网格的线条密度
grid_density = np.array([[len(grid_vectors[i, j]) for j in range(grid_size // grid_step)] for i in range(grid_size // grid_step)])

# 找出高密度区域和低密度区域的阈值
high_density_threshold = np.percentile(grid_density, 90)
low_density_threshold = np.percentile(grid_density, 10)

# 选取高密度和低密度区域
high_density_coords = np.argwhere(grid_density >= high_density_threshold)
low_density_coords = np.argwhere(grid_density <= low_density_threshold)

KeyboardInterrupt: 

In [2]:
# 随机采样高密度区域的网格20%
np.random.seed(0)
high_density_samples = high_density_coords[np.random.choice(len(high_density_coords), len(high_density_coords) // 2, replace=False)]

In [20]:
# 两个网格Jaccard相似系数
def jaccard_similarity(vec1, vec2):
    vec1 = np.array(list(vec1))
    vec2 = np.array(list(vec2))
    intersection = len(set(vec1).intersection(set(vec2)))
    union = len(set(vec1).union(set(vec2)))
    return intersection / union if union > 0 else 0

def calculate_neighborhood_jaccard(sampled_coords, grid_vectors):
    jaccard_scores = {}
    for coord in sampled_coords:
        y, x = coord
        sampled_set = grid_vectors[y, x]
        neighbors = []
        
        # 获取邻近的网格
        for dy in [-1, 0, 1]:
            for dx in [-1, 0, 1]:
                if dy == 0 and dx == 0:
                    continue
                ny, nx = y + dy, x + dx
                if 0 <= ny < grid_vectors.shape[0] and 0 <= nx < grid_vectors.shape[1]:
                    neighbors.append((ny, nx))
        
        # 计算 Jaccard 相似度并存储
        for ny, nx in neighbors:
            neighbor_set = grid_vectors[ny, nx]
            jaccard = jaccard_similarity(sampled_set, neighbor_set)
            jaccard_scores[(y, x, ny, nx)] = jaccard
            
    return jaccard_scores

high_density_jaccard = calculate_neighborhood_jaccard(high_density_samples, grid_vectors)
# low_density_samples = sample_representative(low_density_coords, grid_vectors)

# high_density_samples 和 low_density_samples 现在包含了高密度和低密度区域的代表网格


In [27]:
num_samples = len(high_density_samples)
distance_matrix = np.ones((num_samples, num_samples))

# Step 3: 计算采样网格之间的距离
all_samples = high_density_samples

for i in range(num_samples):
    for j in range(i + 1, num_samples):
        coord1 = all_samples[i]
        coord2 = all_samples[j]
 
        y1, x1 = coord1[0], coord1[1]
        y2, x2 = coord2[0], coord2[1]
        
        jaccard_key1 = (y1, x1, y2, x2)
        jaccard_key2 = (y2, x2, y1, x1)
        
        if jaccard_key1 in high_density_jaccard:
            jaccard = high_density_jaccard[jaccard_key1]
            distance_matrix[i, j] = 1 - jaccard
            distance_matrix[j, i] = 1 - jaccard


In [29]:
non_one_count = np.sum(distance_matrix != 1.0)
non_one_count

102